### Scorecard Automation

#### General Outline

- First find **offset**
- Check date
- Read tables one by one
- check for missing tables
- Check data in each tabel
- Check for missing data
- Check if charts exist
- Validate data with rnramd

___

**Write function to determine which sheet contains which data**


#### List of offsets

> Network Summary Heading : 3

> Bing O&O Core Browse:

---

**Make these global vars so that they can be changed anytime**



In [ ]:
x_offsets = [0, 21, 29, 1, 19, 27, 35, -1, -1, -1, 1, -14, -14]

y_offsets = [6, 6, 6, 18, 18, 18, 18, -1, -1, -1, 61, 75, 89]
widths = [7, 7, 7, 5, 5, 5, 2, -1, -1, -1, 12, 25, 25]

extra_paddings = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 13]

heights = [10, 10, 10, 8, 8, 8, 8, -1, -1, -1, 12, 12, 12]


# make this into a list for each adv table
max_allowable_perc_adv = 1000

segment_names = ['Strategic/Tier 1', 'Mid Market/Tier 2', 'Scale/Tier 3', 'Channel Partner']

if not(len(x_offsets) == len(y_offsets) == len(widths) == len(heights)):
    print ("Table metadata not consistent!")
    raise InterruptedError

#### RNRAMD Config Properties
---

Follow similar structure to NWSC

In [ ]:
rnr_x_offsets = [0]

rnr_y_offsets = [60]
rnr_widths = [5]

rnr_extra_paddings = [0]

rnr_heights = [7]


# make this into a list for each adv table
rnr_max_allowable_perc_adv = 1000

rnr_segment_names = ['Strategic/Tier 1', 'Mid Market/Tier 2', 'Scale/Tier 3', 'Channel Partner']

if not(len(rnr_x_offsets) == len(rnr_y_offsets) == len(rnr_widths) == len(rnr_heights)):
    print ("Table metadata not consistent!")
    raise InterruptedError

#### Setting Development mode

In [ ]:
import platform

print (platform.system())

dev_mode = platform.system()

if dev_mode == "Linux":
    root_path = "/media/shaunak/New Volume/my_projects/data science updated/python for data science/Scorecard Automate/data/"
elif dev_mode == "Windows":
    root_path = "D:/data_science-master/python for data science/Scorecard Automate/data/"


In [ ]:
import openpyxl
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib import style

import numpy as np

import quandl
import datetime

wb = openpyxl.load_workbook(filename=root_path + 'data.xlsx', 
                   read_only=True)
sheet_nw_sc = wb['Sheet4']

data_rows = []
for row in sheet_nw_sc['W10':'AC18']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)

    
nw_sc_df = pd.DataFrame(data_rows)

nw_sc_df.set_index(0, inplace=True)

print (nw_sc_df)

"""
for row in sheet_nw_sc.iter_rows(min_row=21,min_col=23, max_col=30, max_row=25):
    for cell in row:
        print cell.value
"""


wb = openpyxl.load_workbook(filename=root_path + 'data.xlsx', 
                   read_only=True)
sheet_nw_sc = wb['Sheet6']

data_rows = []
for row in sheet_nw_sc['A74':'E80']:
    data_cols = []
    for cell in row:
        data_cols.append(cell.value)
    data_rows.append(data_cols)

    
nw_sc_df = pd.DataFrame(data_rows)

nw_sc_df.set_index(0, inplace=True)

print (nw_sc_df)



#### Read in the NWSC file + determining offset

In [ ]:
offset = 0

base_cell = 'W3'

network_sc_date_as_date = ""

workbook = openpyxl.load_workbook(filename = root_path + 'data3.xlsx', 
                   read_only=False)

network_sc_sheet = workbook['Sheet4']


print (network_sc_sheet.merged_cell_ranges)

first_heading = str(network_sc_sheet['W3'].value)



print (first_heading.strip('-'))


if first_heading is None or first_heading.split('-')[0].strip() != 'SEARCH BG SCORECARD':
    # print first_heading.split('-')[0].strip()
    print ("offset to be changed")
    
    first_heading = str(network_sc_sheet['W14'].value)
    
    if first_heading.split('-')[0].strip() == 'SEARCH BG SCORECARD':
        offset = 11
        print ("Offset changed")

#### Offset changed at this step -- Move to separate function

#### Validating the date

In [ ]:
def validate_date():
    global offset
    global base_cell
    global network_sc_sheet
    global network_sc_date_as_date
    # base cell = W3
    base_cell_num = int(base_cell[1:len(base_cell)])
    base_cell_num += offset
    
    cell_ref = base_cell[0] + str(base_cell_num)
    
    print (cell_ref)
    
    # Modifying the base cell
    
    base_cell = cell_ref
    
    first_heading = network_sc_sheet[cell_ref].value
    
    try:
        network_sc_date = first_heading.split('-')[1]
        network_sc_date = network_sc_date.strip()
        
        print (network_sc_date)
        
        network_sc_date_as_date = datetime.datetime.strptime(network_sc_date, "%B %d, %Y").date()
        
        # print network_sc_date_as_date
        
        current_date = datetime.date.today()
        
        # print datetime.date.today()
        
        # print network_sc_date_as_date + datetime.timedelta(days = 33) == datetime.date.today()
        
        days_lag = abs(current_date - network_sc_date_as_date).days
        
        if days_lag == 0:
            print ("Dates Validated!")
            return True
        else:
            print ("Scorecard is behind by: ", days_lag, "days")
            return False
        
    except IndexError:
        print ("There was some problem parsing the date")
        return False
    
validate_date()

#### Utility Fuction to get cell reference

In [ ]:

def get_cell_ref(increment):
    global base_cell
    base_cell
    base_cell_int = int(base_cell[1:len(base_cell)])
    base_cell_int += increment
    new_cell_ref = base_cell[0] + str(base_cell_int)
    return new_cell_ref

get_cell_ref(3)

#### Utility function to read in cells

In [ ]:
def read_data(cell_ref):
    data = network_sc_sheet[cell_ref].value
    return data

#### Utility function to traverse cells

In [ ]:
list_of_cols = []

for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append(chr(col))

for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('A' + chr(col))
    
for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('B' + chr(col))
    
for col in range(ord('A'), ord('Z')+1):
    list_of_cols.append('C' + chr(col))
    
def traverse_cells(cell, incx, incy, mode=0):
    if mode == 0:
        # w3
        column_number = cell[0]
        row_number = int(cell[1:len(cell)])
    elif mode == 1:
        #ab1
        column_number = cell[0:2]
        row_number = int(cell[2:len(cell)])
        
    index_of_col = list_of_cols.index(column_number)
    index_of_col += incx
    new_col = list_of_cols[index_of_col]
    row_number += incy
    new_cell_ref = new_col + str(row_number)
    
    return new_cell_ref
    
    
    
traverse_cells('BZ1', 5, 28, 1)
    

#### Utility function to get a set of cells and return a pandas DF

In [ ]:
def get_dataframe(start_cell, end_cell):
    data_rows = []
    for row in network_sc_sheet[start_cell: end_cell]:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)
    coverted_df = pd.DataFrame(data_rows) 
    return coverted_df

get_dataframe('AZ20', 'BF29')

In [ ]:
def get_dataframe_rnr(start_cell, end_cell):
    data_rows = []
    for row in sheet_rnr_amd[start_cell: end_cell]:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)
    coverted_df = pd.DataFrame(data_rows) 
    return coverted_df

# get_dataframe_rnr('A74', 'E80')

#### Utility function to extract numeric value from character string

In [ ]:
import types

def convert_to_numeric(number_str):
    print ("String in func:", number_str)
    
    
    if dev_mode == 'linux':
        if type(number_str) == type(b'mini'):
            number_str = number_str.decode()
    list_of_chars = list(str(number_str))
    
    first_num_index = -1
    
    was_int = 0
    
    last_num_index = len(list_of_chars)-1
    for x in range(0, len(list_of_chars)):
        try:
            char_as_num = int(list_of_chars[x])
            was_int = 1
        except ValueError:
            was_int = 0
        if was_int == 1:
            break
            
    first_num_index = x
    # print first_num_index
    
    was_char = 1
    
    for x in range(last_num_index, -1, -1):
        try:
            char_as_num = int(list_of_chars[x])
            was_char = 0
        except ValueError:
            last_num_index -= 1
            was_char = 1
        if was_char == 0:
            break
    last_num_index = x
    # print last_num_index
    
    if (last_num_index < first_num_index):
        return False
    else:
        return float("".join(list_of_chars[first_num_index: last_num_index+1]))
    
convert_to_numeric('93.4¢')    

#### Base cell ref changed at this step -- Move to separate function

#### Dates validated

Check for Network Summary

#### Utility function to check missing and zero values in a list

#### Function to check 1st table data

> Make it resuable

In [ ]:

def check_smart_pricing_data(first_table_df):
    
    # maybe date check should be done here
    
    # first_table_df[3][1] = 0
    # first_table_df[2][3] = '*'
    # first_table_df[2][4] = None
    
    # check for missing data

    # print first_table_df.columns

    for column_index in first_table_df.columns[1:]:
        column_values = first_table_df[column_index][1:].tolist()
        if check_missing_and_zero_values(column_values) is False:
            print ("Smart Pricing Data Missing")
            return False


    print ("Smart Pricing data ok")
    return True





**Note:** When we convert to utf-8 in python 3 it will get converted as a byte string!

In [ ]:
import numbers

def check_missing_and_zero_values(column_list):
    for value in column_list:
        print (value)
        if isinstance(value, numbers.Number) is False:
            if value is None:
                print ("Missing values in First Table")
                return False
            else:

                # not a number 
                # maybe: '$#%$%'
                try:
                    length = len(value)
                    if length < 2:
                        print ("Missing values in First Table")
                        return False
                    else:
                        # problem here: what if u cant encode??
                        if dev_mode == 'Linux':
                            value = value.encode('utf-8')
                        else:
                            value = value.encode('utf-8')
                        if convert_to_numeric(value) is False:
                            print ("Missing values in First Table")
                            return False

                except:
                    print ("Unknown stuff in First Table")
                    return False
        else:
            # number
            number_as_float = float(value)
            if float(value) == 0.0:
                print ("Zero values in First Table")
                return False
                
    return True
                    


In [ ]:
def check_table_data(table_df, table_number):
    
    # maybe date check should be done here
    
    # first_table_df[3][1] = 0
    # first_table_df[2][3] = '*'
    # first_table_df[2][4] = None
    
    height_of_table = heights[table_number - 1]
    width_of_table = widths[table_number - 1]
    
    if table_df.shape == (height_of_table, width_of_table):
        
        # check for missing data
        
        if table_number!= 7:
            table_df.drop([4], axis = 1, inplace = True)
            # print first_table_df.columns
        
        for column_index in table_df.columns[1:]:
            column_values = table_df[column_index][1:].tolist()
            if check_missing_and_zero_values(column_values) is False:
                return False
    

        
        print ("Data ok for table number:", table_number)
        return True
        
    else:
        print ("Dimension of does not match for table number:", table_number)
        return False



In [ ]:
def organic_metrics_validation(first_table_df):
    
    if first_table_df.shape == (10, 6):
        
        required_column = first_table_df[1][1:].tolist()
        print (required_column)
        
        organic_metrics = required_column[:6]
        non_organic_metrics = required_column[6:]
        
        oRev = organic_metrics[5]
        oVol = organic_metrics[4]
        Rev = non_organic_metrics[0]
        Vol = non_organic_metrics[1]
        
        print (oRev, oVol, Rev, Vol)
        
        # test case: oRev = 10.00
        
        oRev = convert_to_numeric(oRev)
        oVol = convert_to_numeric(oVol)
        Rev = convert_to_numeric(Rev)
        Vol = convert_to_numeric(Vol)
        
        print (oRev, oVol, Rev, Vol)
        
        if Rev < oRev or Vol < oVol:
            print ("Organic metrics greater than combined metrics which is not possible")
            return False
        
        return True
        
    else:
        print ("Dimension of First Table does not match")
        return False




#### Function for Network Summary First table

In [ ]:
def network_summary_first_table():
    cell_ref = get_cell_ref(3)
    first_heading = str(read_data(cell_ref))
    # print first_heading
    
    if first_heading.strip() != 'NETWORK SUMMARY':
        print ("Cannot find heading: NETWORK SUMMARY")
        return False
    else:
        # all ok until now
        # try to find second heading
        
        second_heading = str(read_data(get_cell_ref(5)))
        if second_heading.strip() != 'Bing O&O Core Browse':
            print ("Cannot find heading: Bing O&O Core Browse")
            return False
        else:
            # read in the range of cells
            start_cell = get_cell_ref(6)
            # print start_cell
            
            end_cell = traverse_cells(start_cell, widths[0]-1, heights[0]-1, 0)
            # print end_cell
            
            network_summary_first_table_df = get_dataframe(start_cell, end_cell)
            
            
            network_summary_first_table_date = network_summary_first_table_df.loc[0][1]
            network_summary_first_table_date = datetime.datetime.date(network_summary_first_table_date)
            
            # print network_sc_date_as_date
            
            # print network_summary_first_table_date
            
            if (network_sc_date_as_date - network_summary_first_table_date).days == 1:
                print ("Dates validated in First Table")
                
                # check table data
                if check_table_data(network_summary_first_table_df, 1) is False:
                    return False
                
                if organic_metrics_validation(network_summary_first_table_df) is False:
                    return False
            
            else:
                print ("Dates not validated in First Table")
                return False
            
            print ("All ok in First Table!!")
            
            return True
    

network_summary_first_table()

#### Check Nw Summary Second Table

In [ ]:
def network_summary_second_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[1], y_offsets[1]-1, 0)
    second_heading = str(read_data(cell_ref))
    if second_heading.strip() != 'Yahoo O&O':
        print ("Cannot find heading: Yahoo O&O")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[1], y_offsets[1], 0)

        end_cell = traverse_cells(base_cell, x_offsets[1] + widths[1] - 1, y_offsets[1] + heights[1] - 1, 0)

        network_summary_second_table_df = get_dataframe(start_cell, end_cell)
        network_summary_second_table_date = network_summary_second_table_df.loc[0][1]
        network_summary_second_table_date = datetime.datetime.date(network_summary_second_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_second_table_date).days == 1:
            print ("Dates validated in Second Table")

            # check table data
            if check_table_data(network_summary_second_table_df, 2) is False:
                return False

        else:
            print ("Dates not validated in Second Table")
            return False

        print ("All ok in Second Table!!")

        return True


network_summary_second_table()



#### Check NW Summary Third Table (Smart Pricing)

In [ ]:
def network_summary_third_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[2], y_offsets[2]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    if second_heading.strip() != 'Yahoo Syndication':
        print ("Cannot find heading: Yahoo Syndication")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[2], y_offsets[2], 0)

        end_cell = traverse_cells(base_cell, x_offsets[2] + widths[2] - 1, y_offsets[2] + heights[2] - 1, 0)
        
        print (start_cell, end_cell)

        network_summary_third_table_df = get_dataframe(start_cell, end_cell)
        network_summary_third_table_date = network_summary_third_table_df.loc[0][1]
        network_summary_third_table_date = datetime.datetime.date(network_summary_third_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_third_table_date).days == 1:
            print ("Dates validated in Third Table")

            # check table data
            if check_table_data(network_summary_third_table_df, 3) is False:
                if check_smart_pricing_data(network_summary_third_table_df) is False:
                    print ("Samrt Pricing Data not available")
                    return False
                else:
                    print ("Missing values in Third Table")
                    return False
        else:
            print ("Dates not validated in Third Table")
            return False

        print ("All ok in Third Table!!")

        return True


network_summary_third_table()



In [ ]:
def network_summary_fourth_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[3], y_offsets[3]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'AOL O&O Guaranty (PC Only)':
        print ("Cannot find heading: AOL O&O Guaranty (PC Only)")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[3], y_offsets[3], 0)

        end_cell = traverse_cells(base_cell, x_offsets[3] + widths[3] - 1, y_offsets[3] + heights[3] - 1, 0)
        
        print (start_cell, end_cell)

        network_summary_fourth_table_df = get_dataframe(start_cell, end_cell)
        network_summary_fourth_table_date = network_summary_fourth_table_df.loc[0][1]
        network_summary_fourth_table_date = datetime.datetime.date(network_summary_fourth_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_fourth_table_date).days == 1:
            print ("Dates validated in Fourth Table")

            # check table data
            if check_table_data(network_summary_fourth_table_df,4) is False:
                return False
        else:
            print ("Dates not validated in Fourth Table")
            return False

        print ("All ok in Fourth Table!!")

        return True


network_summary_fourth_table()



In [ ]:
def network_summary_fifth_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[4], y_offsets[4]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'AOL O&O':
        print ("Cannot find heading: AOL O&O")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[4], y_offsets[4], 0)

        end_cell = traverse_cells(base_cell, x_offsets[4] + widths[4] - 1, y_offsets[4] + heights[4] - 1, 0)
        
        print (start_cell, end_cell)

        network_summary_fifth_table_df = get_dataframe(start_cell, end_cell)
        network_summary_fifth_table_date = network_summary_fifth_table_df.loc[0][1]
        network_summary_fifth_table_date = datetime.datetime.date(network_summary_fifth_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_fifth_table_date).days == 1:
            print ("Dates validated in Fifth Table")

            # check table data
            if check_table_data(network_summary_fifth_table_df, 5) is False:
                return False
        else:
            print ("Dates not validated in Fifth Table")
            return False

        print ("All ok in Fifth Table!!")

        return True


network_summary_fifth_table()

In [ ]:
def network_summary_sixth_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[5], y_offsets[5]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'AOL Syndication':
        print ("Cannot find heading: AOL Syndication")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[5], y_offsets[5], 0)

        end_cell = traverse_cells(base_cell, x_offsets[5] + widths[5] - 1, y_offsets[5] + heights[5] - 1, 0)
        
        print (start_cell, end_cell)

        network_summary_sixth_table_df = get_dataframe(start_cell, end_cell)
        network_summary_sixth_table_date = network_summary_sixth_table_df.loc[0][1]
        network_summary_sixth_table_date = datetime.datetime.date(network_summary_sixth_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_sixth_table_date).days == 1:
            print ("Dates validated in Fifth Table")

            # check table data
            if check_table_data(network_summary_sixth_table_df, 6) is False:
                return False
        else:
            print ("Dates not validated in Fifth Table")
            return False

        print ("All ok in Fifth Table!!")

        return True


network_summary_sixth_table()

In [ ]:
def network_summary_seventh_table():
    
    cell_ref = traverse_cells(base_cell, x_offsets[6], y_offsets[6]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'AOL (O&O and Syndication)':
        print ("Cannot find heading: AOL Syndication")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[6], y_offsets[6], 0)

        end_cell = traverse_cells(base_cell, x_offsets[6] + widths[6] - 1, y_offsets[6] + heights[6] - 1, 0)
        
        print (start_cell, end_cell)

        network_summary_seventh_table_df = get_dataframe(start_cell, end_cell)
        network_summary_seventh_table_date = network_summary_seventh_table_df.loc[0][1]
        network_summary_seventh_table_date = datetime.datetime.date(network_summary_seventh_table_date)

        # print network_sc_date_as_date

        # print network_summary_first_table_date

        if (network_sc_date_as_date - network_summary_seventh_table_date).days == 1:
            print ("Dates validated in Fifth Table")

            # check table data
            if check_table_data(network_summary_seventh_table_df, 7) is False:
                return False
        else:
            print ("Dates not validated in Seventh Table")
            return False

        print ("All ok in Seventh Table!!")

        return True


network_summary_seventh_table()

> Network Summary In-place validation done

#### Advertiser Metrics Validation

- Check if each segment is there
- No missing values
- No 0 values in performing 1st 2 segments
- No abrubt % changes

Note: special chars might come up in Adv names

#### Utility Function: Custom Error Generation Message

In [ ]:
def generate_error_message(table_number, column_number, adv_number):
    
    segment_number = int(((column_number+3)/3)-1)
    
    
    if (column_number+3)%3 == 0:
         
        print ("Check Segement: ", segment_names[segment_number], "Check Column: Customer, Advertiser Number:", adv_number)
    elif (column_number+3)%3 == 1:
        print ("Check Segement: ", segment_names[segment_number], "Check Column: Spend, Advertiser Number:", adv_number)
    else:
        print ("Check Segement: ", segment_names[segment_number], "Check Column: Spend WoW, Advertiser Number:", adv_number)
        
    return
    

In [ ]:
import numbers

def check_missing_values(column_list, column_number, table_number):
    
    # Modify these functions to help know where EXACTLY error occured
    
    # Missing value in Customer:Adv no:5
    
    
    print ("Col list in func:", column_list)
    
    
    adv_number = 0
    for value in column_list:
        print (value)
        print (column_number, adv_number)
        if isinstance(value, numbers.Number) is False:
            if value is None:
                print ("Missing Value in Table Number:", table_number)
                generate_error_message(table_number, column_number, adv_number)
                return False
            else:
        
                try:
                    length = len(value)
                    if length < 2:
                        print ("Missing Value in Table Number:", table_number)
                        generate_error_message(table_number, column_number, adv_number)
                        return False
                except:
                    print ("Unknown Stuff in Table Number:", table_number)
                    return False
        adv_number += 1
                
    return True


import numbers

def check_zero_values(column_list, column_number, table_number):
    
    adv_number = 1
    for value in column_list:
        print (value)
        print (column_number, adv_number)
        if isinstance(value, numbers.Number) is False:
            
            # not a number: not possible
            print ("Advertiser Metric Spend should be numerical")
            generate_error_message(table_number, column_number, adv_number)
            return False
            
        else:
            # number
            number_as_float = float(value)
            if number_as_float == 0.0:
                print ("Zero value in First 2 Segemnts!")
                generate_error_message(table_number, column_number, adv_number)
                if table_number < 3:
                    return False
            
        adv_number += 1
                
    return True

def check_abrupt_values(column_list, column_number, table_number):
    
    adv_number = 1
    for value in column_list:
        print (value)
        if isinstance(value, numbers.Number) is False:
            
            # not a number: not possible
            print ("Advertiser Metric Spend WoW should be numerical")
            generate_error_message(table_number, column_number, adv_number)
            return False
            
        else:
            # number
            number_as_float = float(value)
            if number_as_float*100 > 200:
                print ("Spend WoW % > Allowable %:", max_allowable_perc_adv)
                generate_error_message(table_number, column_number, adv_number)
                
                # dont return False , just alert user
                # return False
            
        adv_number += 1
                
    return True



def validate_advertiser_metrics(adv_df, table_number):
    
    # check if each segement is there of not
    
    # test case: adv_df[0][0] = ""
    try:
        
        # get segment names
        
        first_segment = adv_df[0][0].strip()
        second_segment = adv_df[3][0].strip()
        third_segement = adv_df[6][0].strip()
        fourth_segment = adv_df[9][0].strip()
    
        print (first_segment, second_segment, third_segement, fourth_segment)

        # validate segment names
        
        if first_segment != segment_names[0] or second_segment != segment_names[1] or third_segement != segment_names[2] or fourth_segment != segment_names[3]:
            print ("Segment Names not matching in Advertiser Metrics Table Number:", table_number)
            return False
    except:
        print("Unknown stuff in Advertiser Metrics Table Number:", table_number)
        return False
    
    # check for misisng values for entire df
    
    for column_index in adv_df.columns:
            column_values = adv_df[column_index][1:].tolist()
            if check_missing_values(column_values, column_index, table_number) is False:
                return False
            
    # check for 0 values in first 2 segments
    
        # check for 0 value in spend
        # check for abrupt % change in Spend WoW
        
        
    # 1, 4, 

    
    
    for column_index in range(1, 5, 3):
        column_values = adv_df[column_index][2:].tolist()
        if check_zero_values(column_values, column_index, table_number) is False:
            return False
        
    # 2, 5, 8, 11
        
    for column_index in range(2, 12, 3):
        column_values = adv_df[column_index][2:].tolist()
        if check_abrupt_values(column_values, column_index, table_number) is False:
            return False
        
    
    return True
    

In [ ]:
def top_spend_performing_advertisers():
    cell_ref = traverse_cells(base_cell, x_offsets[10], y_offsets[10]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'Top Spend Performing Customers':
        print ("Cannot find heading: Top Spend Performing Customers")
        return False
    else:
    
        # read in the range of cells
        start_cell = traverse_cells(base_cell, x_offsets[10], y_offsets[10], 0)

        end_cell = traverse_cells(base_cell, x_offsets[10] + widths[10] - 1, y_offsets[10] + heights[10] - 1, 0)
        
        print (start_cell, end_cell)

        top_spend_performing_advertisers_df = get_dataframe(start_cell, end_cell)
        
        # return top_spend_performing_advertisers_df
        
        if validate_advertiser_metrics(top_spend_performing_advertisers_df, 1) is False:
            return False
        return True

top_spend_performing_advertisers()

In [ ]:
def top_spend_DoD_advertisers():
    cell_ref = traverse_cells(base_cell, x_offsets[11], y_offsets[11]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'Top Spend Day Over Day Gainers Customers':
        print ("Cannot find heading: Top Spend Day Over Day Gainers Customers")
        return False
    else:
    
        # read in the range of cell1
        start_cell = traverse_cells(base_cell, x_offsets[11], y_offsets[11], 0)

        end_cell = traverse_cells(base_cell, x_offsets[11] + widths[11] - 1, y_offsets[11] + heights[11] - 1, 0)
        
        print (start_cell, end_cell)

        top_spend_DoD_gainer_advertisers_df = get_dataframe(start_cell, end_cell)
        
        for x in range(1, extra_paddings[11]+1):
            del top_spend_DoD_gainer_advertisers_df[x]
        
        
        actual_width = widths[11] - extra_paddings[11]
        
        top_spend_DoD_gainer_advertisers_df.columns = list(range(actual_width))
        
        # we get the clean df at this stage
                
        if validate_advertiser_metrics(top_spend_DoD_gainer_advertisers_df, 2) is False:
            return False
        return True

top_spend_DoD_advertisers()

In [ ]:
def top_spend_DoD_loser_advertisers():
    cell_ref = traverse_cells(base_cell, x_offsets[12], y_offsets[12]-1, 0)
    second_heading = str(read_data(cell_ref))
    
    print(second_heading)
    
    if second_heading.strip() != 'Top Spend Day Over Day Losers Customers':
        print ("Cannot find heading: Top Spend Day Over Day Losers Customers")
        return False
    else:
    
        # read in the range of cell1
        start_cell = traverse_cells(base_cell, x_offsets[12], y_offsets[12], 0)

        end_cell = traverse_cells(base_cell, x_offsets[12] + widths[12] - 1, y_offsets[12] + heights[12] - 1, 0)
        
        print (start_cell, end_cell)

        top_spend_DoD_loser_advertisers_df = get_dataframe(start_cell, end_cell)
        
        for x in range(1, extra_paddings[12]+1):
            del top_spend_DoD_loser_advertisers_df[x]
        
        
        actual_width = widths[12] - extra_paddings[12]
        
        top_spend_DoD_loser_advertisers_df.columns = list(range(actual_width))
        
        # we get the clean df at this stage
                        
        if validate_advertiser_metrics(top_spend_DoD_loser_advertisers_df, 3) is False:
            return False
        return True

top_spend_DoD_loser_advertisers()

#### Reading RNRAMD

In [ ]:

wb = openpyxl.load_workbook(filename=root_path + 'data.xlsx', 
                   read_only=True)
sheet_rnr_amd = wb['Sheet6']

#### Determining Offset for RNRAMD

In [ ]:
offset_rnr = 0

base_cell_rnr = 'A14'


first_heading = str(sheet_rnr_amd[base_cell].value)


print(first_heading)

if first_heading is None or first_heading.split('-')[0].strip() != 'Bing O&O Core Organic (PC Browse & Tablet) Incl. Kindle':
    # print first_heading.split('-')[0].strip()
    print ("offset to be changed")
    
else:
    print("No need to change offset")

In [ ]:
def read_bing_rnr_table():
    start_cell = traverse_cells(base_cell_rnr, rnr_x_offsets[0], rnr_y_offsets[0], 0)

    end_cell = traverse_cells(base_cell_rnr, rnr_x_offsets[0] + rnr_widths[0] - 1, rnr_y_offsets[0] + rnr_heights[0] - 1, 0)

    print (start_cell, end_cell)

    rnr_bing_df = get_dataframe_rnr(start_cell, end_cell)
    
    return rnr_bing_df
    
    
read_bing_rnr_table()